# Loading Data

In [66]:
import glob

langs = ['en', 'de', 'fi', 'ja']

tasks = ['negation1', 'negation2', 
         'numerals1', 'numerals2', 
         'spatial1', 'spatial2', 
         'temporal1', 'temporal2', 
         'comparative1', 'comparative2',
         'counters']

files = [{'task': task, 'lang': lang, 'path': file} for lang in langs for task in tasks for file in glob.glob(f'/home/norrman/GitHub/multi-morph-checklist/export/*{task}*/{lang}.txt')]
for file in files:
    if '_fam/' in file['path']:
        file['polite'] = 'fam'
    elif '_norm/' in file['path']:
        file['polite'] = 'norm'
    elif '_form/' in file['path']:
        file['polite'] = 'form'
    else:
        file['polite'] = ''

files

[{'task': 'negation1',
  'lang': 'en',
  'path': '/home/norrman/GitHub/multi-morph-checklist/export/negation1/en.txt',
  'polite': ''},
 {'task': 'negation2',
  'lang': 'en',
  'path': '/home/norrman/GitHub/multi-morph-checklist/export/negation2/en.txt',
  'polite': ''},
 {'task': 'numerals1',
  'lang': 'en',
  'path': '/home/norrman/GitHub/multi-morph-checklist/export/numerals1/en.txt',
  'polite': ''},
 {'task': 'numerals2',
  'lang': 'en',
  'path': '/home/norrman/GitHub/multi-morph-checklist/export/numerals2/en.txt',
  'polite': ''},
 {'task': 'spatial1',
  'lang': 'en',
  'path': '/home/norrman/GitHub/multi-morph-checklist/export/spatial1/en.txt',
  'polite': ''},
 {'task': 'spatial2',
  'lang': 'en',
  'path': '/home/norrman/GitHub/multi-morph-checklist/export/spatial2/en.txt',
  'polite': ''},
 {'task': 'temporal1',
  'lang': 'en',
  'path': '/home/norrman/GitHub/multi-morph-checklist/export/temporal1/en.txt',
  'polite': ''},
 {'task': 'temporal2',
  'lang': 'en',
  'path': '/h

In [67]:
TARGET_PATTERN = {
    'en': { # {N1} (is/are) {NEG} {P2}, but {P1} is/are.
           'negation1': r".*"
                        r"C\:\s(?:(?P<N1A>\w+)\sand\s(?P<N1B>\w+)|(?P<N1>\w+))\s(?:is|are)\snot\s((?:a|an)\s)?(?P<P2>.+?),\sbut\s(?:(?P<P1A>.+?)\sand\s(?P<P1B>.+?)|(?P<P1>.+?))\s(?:is|are)\."
                        r".*",
                        
            # {N1} (is/are) a {N2}, {P1} are {P2}.
           'negation2': r".*"
                        r"C\:\s(?:(?P<N1A>\w+)\sand\s(?P<N1B>\w+)|(?P<N1>\w+))\s(?:is|are)\s((?:a|an)\s)?(?P<N2>.+?),\s(?:(?P<P1A>.+?)\sand\s(?P<P1B>.+?)|(?P<P1>.+?))\s(?:is|are)\s((?:a|an)\s)?(?P<P2>.+?)\."
                        r".*",
            # C: There are {N1} {N2} and {N3} {N4} {P3}. 
            # Q: How many {P2} in total are {P3}? 
            # A: {P1}.
           'numerals1': r".*"
                        r"C\:\sThere\s(?:is|are)\s(?P<N1A>.+?)\s(?P<N2A>.+?)\sand\s(?P<N1B>.+?)\s(?P<N2B>.+?)\s(?P<P3>.+?)\.\s.*"
                        r"Q\:\sHow\smany\s(?P<P2>.+?)\s.*"
                        r"A:\s(?P<P1>.+?)\."
                        r".*",

            # C: There are {N1A} {P2} and {N1B} {N2} {P3}. Kirk ate {N1C} pineapples. 
            # Q: How many pineapples are on the table? 
            # A: {P1}.
           'numerals2': r".*"
                        r"C\:\sThere\s(?:is|are)\s(?P<N1A>.+?)\s(?P<P2>.+?)\sand\s(?P<N1B>.+?)\s(?P<N2>.+?)\s(?P<P3>.+?)\.\s.*ate\s(?P<N1C>.+?)\s.*"
                        r"A:\s(?P<P1>.+?)\."
                        r".*",

            # C: The {N2} is {N1} and the {P2} are {P1}. 
            # Q: Where are the phones? 
            # A: Next to the shelf.
           'spatial1': r".*"
                       r"C\:\sThe\s(?P<N2>.+?)\s(?:is|are)\s(?P<N1>.+?)\sand\sthe\s(?P<P2>.+?)\s(?:is|are)\s(?P<P1>.+?)\."
                       r".*",
              # C: The {P2} and the {N2} are {P1}. Kirk puts the pens {N1}. 
              # Q: Where is the book? 
              # A: Behind the table.
           'spatial2': r".*"
                       r"C\:\sThe\s(?P<P2>.+?)\sand\sthe\s(?P<N2>.+?)\sare\s(?P<P1>.+?)\.\s\w+\sputs\sthe\s\w+\s(?P<N1>.+?)\."
                       r".*",
               # C: {P1} and {N1} were {N2}, but Lola changed careers and is now a {P2}. 
               # Q: Who is a farmer? 
               # A: Lola.
           'temporal1': r".*"
                        r"C\:\s(?P<P1>.+?)\sand\s(?P<N1>.+?)\swere\s(?P<N2>.+?),\s.*is\snow\san?\s(?P<P2>.+?)\."
                        r".*",
               # C: {P1} and {N1} are {N2}, but Audrey will change careers and will be a {P2}. 
               # Q: Who will be a policeman? 
               # A: Audrey.
           'temporal2': r".*"
                        r"C\:\s(?P<P1>.+?)\sand\s(?P<N1>.+?)\sare\s(?P<N2>.+?),\s.*will\sbe\san?\s(?P<P2>.+?)\.\sQ"
                        r".*",
               # C: {N1} is {N2} than {P1}. 
               # Q: Who is {P2}? 
               # A: Lola.
           'comparative1': r".*"
                           r"C\:\s(?P<N1>.+?)\sis\s(?P<N2>.+?)\sthan\s(?P<P1>.+?)\.\sQ\:\sWho\sis\s(?P<P2>.+?)\?"
                           r".*",
               # C: If the {P1} was more expensive, Lola would {P2} it. If the {N1} was cheaper, Lola would rent it. 
               # Q: What would Lola rent if it was less cheap? 
               # A: The house.
           'comparative2': r".*"
                           r"C\:\sIf\sthe\s(?P<P1>.+?)\swas\s.*would\s(?P<P2>.+?)\sit\.\sIf\sthe\s(?P<N1>.+?)\swas\s"
                           r".*",
           },
    'de': { # {N1} (ist/sind) {NEG} {P2}, aber {P1} ist/sind es.'
           'negation1': r".*"
                        r"C\:\s(?:(?P<N1A>\w+)\sund\s(?P<N1B>\w+)|(?P<N1>\w+))\s(?:ist|sind)\skeine?\s(?P<P2>.+?),\saber\s(?:(?P<P1A>.+?)\sund\s(?P<P1B>.+?)|(?P<P1>.+?))\s(?:ist|sind)\ses\."
                        r".*",

            # {N1} ist/sind ein {N2} und {P1} sind {P2}.
           'negation2': r".*"
                        r"C\:\s(?:(?P<N1A>\w+)\sund\s(?P<N1B>\w+)|(?P<N1>\w+))\s(?:ist|sind)\s(eine?\s)?(?P<N2>.+?)\sund\s(?:(?P<P1A>.+?)\sund\s(?P<P1B>.+?)|(?P<P1>.+?))\s(?:ist|sind)\s(eine?\s)?(?P<P2>.+?)\."
                        r".*",

            # C: Es gibt {N1} {N2} und {N3} {N4} {P3}. 
            # Q: Wie viele {P2} liegen insgesamt {P3}? 
            # A: {P1}.
           'numerals1': r".*"
                        r"C\:\sEs\sgibt\s(?P<N1A>.+?)\s(?P<N2A>.+?)\sund\s(?P<N1B>.+?)\s(?P<N2B>.+?)\s(?P<P3>.+?)\.\s.*"
                        r"Q\:\sWie\sviele\s(?P<P2>.+?)\s.*"
                        r"A:\s(?P<P1>.+?)\."
                        r".*",
            # C: Es gibt {N1A} {P2} und {N1B} {N2} {P3}. Katja aß {N1C} Wassermelone. 
            # Q: Wie viele Wassermelonen gibt es auf dem Tisch? 
            # A: {P1}.
           'numerals2': r".*"
                        r"C\:\sEs\sgibt\s(?P<N1A>.+?)\s(?P<P2>.+?)\sund\s(?P<N1B>.+?)\s(?P<N2>.+?)\s(?P<P3>.+?)\.\s.*aß\s(?P<N1C>.+?)\s.*"
                        r"A:\s(?P<P1>.+?)\."
                        r".*",
            # C: Der {N2} liegt {N1} und die {P2} liegen {P1}. 
            # Q: Wo sind die Bücher? 
            # A: Vor dem Regal.
           'spatial1': r".*"
                       r"C\:\s(?:Der|Die|Das)\s(?P<N2>.+?)\s(?:liegt|liegen)\s(?P<N1>.+?)\sund\s(?:der|die|das)\s(?P<P2>.+?)\s(?:liegt|liegen)\s(?P<P1>.+?)\."
                       r".*",
              # C: Das {P2} und die {N2} liegen {P1}. Judith legt die Papiere {N1}. 
              # Q: Wo ist das Buch? 
              # A: Vor dem Sofa.
           'spatial2': r".*"
                       r"C\:\s(?:Der|Die|Das)\s(?P<P2>.+?)\sund\s(?:der|die|das)\s(?P<N2>.+?)\s(?:liegt|liegen)\s(?P<P1>.+?)\.\s\w+\slegt\s(?:der|die|das|den)\s\w+\s(?P<N1>.+?)\."
                       r".*",
               # C: {P1} und {N1} waren {N2}, aber Klaus hat den Beruf gewechselt und wird {P2}. 
               # Q: Wer ist Manager? 
               # A: Klaus.
           'temporal1': r".*"
                        r"C\:\s(?P<P1>.+?)\sund\s(?P<N1>.+?)\swaren\s(?P<N2>.+?),\s.*und\swird\s(?P<P2>.+?)\."
                        r".*",
               # C: {P1} und {N1} sind {N2}, aber Günter wechselt den Beruf und wird {P2}. 
               # Q: Wer wird Buchhalter? 
               # A: Günter.
           'temporal2': r".*"
                        r"C\:\s(?P<P1>.+?)\sund\s(?P<N1>.+?)\ssind\s(?P<N2>.+?),\s.*und\swird\s(?P<P2>.+?)\."
                        r".*",
               # C: {N1} ist {N2} als {P1}. 
               # Q: Wer ist {P2}? 
               # A: Stefan.
           'comparative1': r".*"
                           r"C\:\s(?P<N1>.+?)\sist\s(?P<N2>.+?)\sals\s(?P<P1>.+?)\.\sQ\:\sWer\sist\s(?P<P2>.+?)\?"
                           r".*",
               # C: Wäre die {P1} älter, würde sie Stefan {P2}. Wäre das {N1} teurer, würde es Stefan mieten. 
               # Q: Was würde Stefan mieten, wäre es weniger neu? 
               # A: Die Wohnung.
           'comparative2': r".*"
                           r"C\:\s\w+\s\w+\s(?P<P1>.+?)\s\w+,\s\w+\s\w+\s\w+\s(?P<P2>.+?)\.\s\w+\s\w+\s(?P<N1>.+?)\s\w+,"
                           r".*",
           },          
    'fi': { # {N1} {NEG} ole {P2} mutta {P1} on/ovat.
           'negation1': r".*"
                        r"C\:\s(?:(?P<N1A>\w+)\sja\s(?P<N1B>\w+)|(?P<N1>\w+))\s(?:ei|eivät)\sole\s(?P<P2>.+?)\smutta\s(?:(?P<P1A>.+?)\sja\s(?P<P1B>.+?)|(?P<P1>.+?))\s(?:on|ovat)\."
                        r".*",

           # {N1} on {N2}, {P1} ovat {P2}. 
           'negation2': r".*"
                        r"C\:\s(?:(?P<N1A>\w+)\sja\s(?P<N1B>\w+)|(?P<N1>\w+))\s(?:on|ovat)(?P<N2>.+?),\s(?:(?P<P1A>.+?)\sja\s(?P<P1B>.+?)|(?P<P1>.+?))\s(?:on|ovat)(?P<P2>.+?)"
                        r".*",
              # C: {P3} on {N1} {N2} ja {N3} {N4}. 
              # Q: Kuinka monta {P2} on {P3} yhteensä? 
              # A: {P1}.
           'numerals1': r".*"
                        r"C\:\s(?P<P3>.+?)\son\s(?P<N1A>.+?)\s(?P<N2A>.+?)\sja\s(?P<N1B>.+?)\s(?P<N2B>.+?)\.\s.*"
                        r"Q\:\sKuinka\smonta\s(?P<P2>.+?)\s.*"
                        r"A:\s(?P<P1>.+?)\."
                        r".*",
              # C: {P3} on {N1A} {P2} ja {N1B} {N2}. Ylvä söi {N1C} vesimelonia. 
              # Q: Kuinka monta vesimelonia on pöydällä? '
              # A: {P1}.        
           'numerals2': r".*"
                        r"C\:\s(?P<P3>.+?)\son\s(?P<N1A>.+?)\s(?P<P2>.+?)\sja\s(?P<N1B>.+?)\s(?P<N2>.+?)\.\s.*söi\s(?P<N1C>.+?)\s.*"
                        r"A:\s(?P<P1>.+?)\."
                        r".*",
              # C: {N2} on {N1} ja {P2} {P1}. 
              # Q: Missä ovat paperit? 
              # A: Tuolin edessä.
           'spatial1': r".*"
                       r"C\:\s(?P<N2>.+?)\s(?:on|ovat)\s(?P<N1>.+?)\sja\s(?P<P2>.+?)\s(?P<P1>.+?)\."
                       r".*",
              # C: {P2} ja {N2} ovat {P1}. Okko laittaa paperit {N1}. 
              # Q: Missä on kynä? 
              # A: Hyllyn päällä.
           'spatial2': r".*"
                       r"C\:\s(?P<P2>.+?)\sja\s(?P<N2>.+?)\s(?:on|ovat)\s(?P<P1>.+?)\.\s\w+\slaittaa\s\w+\s(?P<N1>.+?)\."
                       r".*",
               # C: {P1} ja {N1} olivat {N2} mutta Hellä vaihtoi uraa ja on nyt {P2}. 
               # Q: Kuka on lakimies? 
               # A: Hellä.
           'temporal1': r".*"
                        r"C\:\s(?P<P1>.+?)\sja\s(?P<N1>.+?)\solivat\s(?P<N2>.+?)\smutta\s.*on\snyt\s(?P<P2>.+?)\."
                        r".*",
               # C: {P1} ja {N1} ovat {N2} mutta Jaakko vaihtaa uraa ja tulee olemaan {P2}. 
               # Q: Kuka tulee olemaan kirjanpitäjä? 
               # A: Jaakko.
           'temporal2': r".*"
                        r"C\:\s(?P<P1>.+?)\sja\s(?P<N1>.+?)\sovat\s(?P<N2>.+?)\smutta\s.*ja\stulee\solemaan\s(?P<P2>.+?)\."
                        r".*",
               # C: {N1} on {N2} kuin {P1}. 
               # Q: Kumpi on {P2}? 
               # A: Okko.
           'comparative1': r".*"
                           r"C\:\s(?P<N1>.+?)\son\s(?P<N2>.+?)\skuin\s(?P<P1>.+?)\.\sQ\:\sKumpi\son\s(?P<P2>.+?)\?"
                           r".*",
               # C: Jos kirja olisi pienempi, Jaakko {P2} sitä. Jos {N1} olisi isompi, Jaakko käyttäisi sitä. 
               # Q: Mitä Jaakko käyttäisi jos se olisi vähemmän iso? 
               # A: {P1}.
           'comparative2': r".*"
                           r"C\:\sJos\s\w+.*,\s\w+\s(?P<P2>.+?)\s(?:sitä|sen).\sJos\s(?P<N1>.+?)\solisi\s.*A:\s(?P<P1>.+?)\."
                           r".*",
           },
    'ja': {# {N1}は{P2}(じゃないけど/ではないですが/ございませんが)、{P1}はそうです。
           'negation1': r".*"
                        r"C\:\s(?:(?P<N1A>.+)と(?P<N1B>.+)|(?P<N1>.+))は(?P<P2>.+?)(?:じゃないけど|ではないですが|ではございませんが)、(?:(?P<P1A>.+)と(?P<P1B>.+)|(?P<P1>.+))はそう(?:です|だ|でございます)。"
                        r".*",

           # {N1}は{N2}、{P1}は{P2}だ。
           'negation2': r".*"
                        r"C\:\s(?:(?P<N1A>.+)と(?P<N1B>.+)|(?P<N1>.+))は(?P<N2>.+?)、(?:(?P<P1A>.+)と(?P<P1B>.+)|(?P<P1>.+))は(?P<P2>.+?)(?:だ|です|でございます)。\s?Q\:"
                        r".*",
       
              # C: {P3}に{N1}の{N2}と{N3}の{N4}がある。
              # Q: {P3}にはいくつの{P2}がある？
              # A: {P1}。
           'numerals1': r".*"
                        r"C\:\s(?P<P3>.+?)に(?P<N1A>.+?)の(?P<N2A>.+?)と(?P<N1B>.+?)の(?P<N2B>.+?)が(?:ある|あります|ございます)。.*"
                        r"Q\:\s.*にはいくつの(?P<P2>.+?)が(?:ある|あります|ございます).*"
                        r"A:\s(?P<P1>.+?)(?:です|ございます)?。"
                        r".*",

              # C: {P3}に{N1A}の{P2}と{N1B}の{N2}がございます。富美子はりんごを二個召し上がりました。
              # Q: テーブルの上にりんごはいくつございますか。
              # A: {P1}ございます。        
           'numerals2': r".*"
                        r"C\:\s(?P<P3>.+?)に(?P<N1A>.+?)の(?P<P2>.+?)と(?P<N1B>.+?)の(?P<N2>.+?)が(?:ある|あります|ございます)。.*を(?P<N1C>.+?)(?:食べた|食べました|召し上がりました)。.*"
                        r"A:\s(?P<P1>.+?)(?:です|ございます)?。"
                        r".*",
            # C: {N2}は{N1}、{P2}は{P1}にございます。
            # Q: 紙はどこにございますかか。
            # A: 机の上にございます。
           'spatial1': r".*"
                       r"C\:\s(?P<N2>.+?)は(?P<N1>.+?)、(?P<P2>.+?)は(?P<P1>.+?)に(?:ある|あります|ございます)。"
                       r".*",
            # C: {P2}と{N2}は{P1}にございます。富美子様は電話を{N1}に置かれました。
            # Q: 本はどこにございますか。
            # A: ソファの下にございます。
           'spatial2': r".*"
                       r"C\:\s(?P<P2>.+?)と(?P<N2>.+?)は(?P<P1>.+?)(?:だ|にあります|にございます)。.*を(?P<N1>.+?)に(?:置いた|置きます|置かれました)。"
                       r".*",
            # C: {P1}と{N1}は{N2}でいらっしゃいましたが、笑子様はご転職されて今は{P2}でいらっしゃいます。 
            # Q: 誰が歌手でいらっしゃいますか。
            # A: 笑子様でございます。
           'temporal1': r".*"
                        r"C\:\s(?P<P1>.+?)と(?P<N1>.+?)は(?P<N2>.+?)(?:だった|でした|でいらっしゃいました).*今は(?P<P2>.+?)(?:。|です|でいらっしゃいます)"
                        r".*",
            # C: {P1}と{N1}は{N2}でいらっしゃいますが、春花様はご転職して{P2}になられます。
            # Q: 警官には誰がなられますか。
            # A: 春花様でございます。
           'temporal2': r".*"
                        r"C\:\s(?P<P1>.+?)と(?P<N1>.+?)は(?P<N2>.+?)(?:だけど|ですが|でいらっしゃいますが)、.*転職して(?P<P2>.+?)(?:になる|になります|になられます)。"
                        r".*",
            # C: {N1}は{P1}より{N2}でいらっしゃいます。
            # Q: 誰が{P2}でいらっしゃいますか？
            # A: 実様でございます。
           'comparative1': r".*"
                           r"C\:\s(?P<N1>.+?)は(?P<P1>.+?)より(?P<N2>.+?)(?:。|だ。|です。|でいらっしゃいます。)"
                           r".*Q\:\s(?:どっち|誰|どちら)が(?P<P2>.+?)(?:？|だ|です|でいらっしゃいます)"
                           r".*",
            # C: もしその{P1}がもっと美しかったならば、龍様はそれを{P2}でしょう。もしその{N1}がもっと大きかったならば、龍様はそれを売るでしょう。
            # Q: 何がもっと醜くなかったならば、龍様はそれを売るでしょうか。
            # A: 船でございます。
           'comparative2': r".*"
                           r"C\:\sもしその(?P<P1>.+?)がもっと.*それを(?P<P2>.+?)(?:だろう|でしょう)。もしその(?P<N1>.+?)がもっと"
                           r".*",
            # C: 棚の上に{N1}の物がありますが、それらは辞書です。
            # Q: 棚の上にいくつの辞書がありますか。
            # A: {P1}です。
            'counters': r".*"
                        r"C\:\s(?P<P2>.+?)に(?P<N1>.+?)の物.*A:\s(?P<P1>.+?)(?:。|です。|ございます。)"
                        r".*",
           },
}

CONJ = {"en": "and",
        "de": "und",
        "fi": "ja",
        "ja": "と"}

NEG = {"en": ("not", "no"),
       "de": ("keine", "kein"),
       "fi": ("ei", "eivät"),
       "ja": ("じゃない", "ではない", 
               "じゃありません", "ではありません", 
               "ございません")}

In [68]:
# NAMES -
#      En - Dennis, Lisa
#      De - Thorsten Marie
#      Fi - Antti, Kati
#      Ja - 昭夫, 文江

# JOBS -
#      En - programmer, therapist
#      De - programmierer, therapeutin
#      Fi - ohjelmoija, terapeutti
#      Ja - プログラマー、療法士

# OBJECT
#      En - bottle, box
#      De - Flasche, Schachtel
#      Fi - pullo, laatikko
#      Ja - ボトル、ボックス

# FRUITS -
#      En - peach, persimmon
#      De - Pfirsich, Dattelpflaume
#      Fi - persikka, persimoni
#      Ja - いちじく、さくらんぼ

# PLACE - 
#      En - drawer, freezer
#      De - Schublade, Gefrierschrank
#      Fi - laatikko, pakastin
#      Ja - 引き出し、冷凍庫

# COMPS -
#      En - wider, thinner
#      De - breiter, dünner
#      Fi - leveämpi, ohuempi
#      Ja - より広い、より薄い

# ACTION -
#      En - discarded
#      De - weggeworfen
#      Fi - olisi hylännyt
#      Ja - 捨てた



ONE_SHOT_PROMPTS = {
                    'en': {'negation1': 'C: Dennis is not a programmer, but Lisa is. Q: Who is an programmer? A: Lisa.', 
                           'negation2': 'C: Dennis is a programmer, Lisa is a therapist. Q: Who is not a programmer? A: Lisa.', 
                           'numerals1': 'C: There are two peaches and six persimmons on the table. Q: How many fruits in total are on the table? A: Eight.', 
                           'numerals2': 'C: There are eight peaches and nine persimmons on the table. Lisa ate two peaches. Q: How many peaches are on the table? A: Six.', 
                           'spatial1': 'C: The bottle is behind the drawer and the box is next to the freezer. Q: Where is the box? A: Next to the freezer.', 
                           'spatial2': 'C: The bottle and the box are behind the drawer. Dennis puts the box on the ground. Q: Where is the bottle? A: Behind the drawer.', 
                           'temporal1': 'C: Dennis and Lisa were writers, but Lisa changed careers and is now a therapist. Q: Who is a therapist? A: Lisa.', 
                           'temporal2': 'C: Dennis and Lisa are writers, but Lisa will change careers and will be a therapist. Q: Who will be a therapist? A: Lisa.', 
                           'comparative1': 'C: Lisa is wider than Dennis. Q: Who is thinner? A: Dennis.', 
                           'comparative2': 'C: If the drawer was wider, Lisa would discard it. If the freezer was thinner, Lisa would discard it. Q: What would Lisa discard if it was less thin? A: The drawer.',
                           }, 
                    'de': {'negation1': 'C: Thorsten ist keine Programmierer, aber Marie ist es. Q: Wer ist Programmierer? A: Lisa.', 
                           'negation2': 'C: Thorsten ist eine Programmierer und Marie ist eine Therapeutin. Q: Wer ist keine Programmierer? A: Marie.', 
                           'numerals1': 'C: Es gibt eine Pfirsich und zwei Dattelpflaume auf dem Tisch. Q: Wie viele Früchte liegen insgesamt auf dem Tisch? A: Drei.', 
                           'numerals2': 'C: Es gibt acht Pfirsich und acht Dattelpflaume auf dem Tisch. Katja aß zwei Pfirsich. Q: Wie viele Pfirsich gibt es auf dem Tisch? A: Sechs.', 
                           'spatial1': 'C: Die Flasche liegt hinter der Schublade und die Schachtel liegen vor dem Gefrierschrank. Q: Wo sind die Schachtel? A: Vor dem Gefrierschrank.', 
                           'spatial2': 'C: Die Flasche und die Schachtel liegen hinter der Schublade. Marie legt die Schachtel auf den Boden. Q: Wo ist die Flasche? A: Hinter der Schublade.', 
                           'temporal1': 'C: Thorsten und Marie waren Programmierer, aber Marie hat den Beruf gewechselt und wird Therapeutin. Q: Wer ist Therapeutin? A: Marie.', 
                           'temporal2': 'C: Thorsten und Marie sind Programmierer, aber Marie wechselt den Beruf und wird Therapeutin. Q: Wer wird Therapeutin? A: Marie.', 
                           'comparative1': 'C: Lisa ist breiter als Thorsten. Q: Wer ist dünner? A: Thorsten.', 
                           'comparative2': 'C: Wäre die Schublade breiter, würde sie Thorsten wegwerfen. Wäre der Gefrierschrank dünner, würde sie Thorsten wegwerfen. Q: Was würde Stefan wegwerfen, wäre es weniger dünn? A: die Schublade.',
                           }, 
                    'fi': {'negation1': 'C: Antti ei ole ohjelmoija mutta Kati on. Q: Kuka on ohjelmoija? A: Kati.', 
                           'negation2': 'C: Antti on ohjelmoija, Kati on terapeutti. Q: Kuka ei ole ohjelmoija? A: Kati.', 
                           'numerals1': 'C: Pöydällä on yksi persikka ja yksi persimoni. Q: Kuinka monta hedelmää on pöydällä yhteensä? A: Kaksi.', 
                           'numerals2': 'C: Pöydällä on seitsemän persikka ja kahdeksan persimoni. Antti söi yhden persikan. Q: Kuinka monta banaania on pöydällä? A: Kuusi.', 
                           'spatial1': 'C: Pullo on kaapin vieressä ja laatikko pakastimen edessä. Q: Missä ovat laatikko? A: Pakastimen edessä.', 
                           'spatial2': 'C: Pullo ja laatikko ovat kaapin päällä. Antti laittaa pullo maahan. Q: Missä on laatikko? A: Pakastimen edessä.', 
                           'temporal1': 'C: Antti ja Kati olivat ohjelmoijia mutta Kati vaihtoi uraa ja on nyt terapeutti. Q: Kuka on terapeutti? A: Kati.', 
                           'temporal2': 'C: Antti ja Kati ovat ohjelmoijia mutta Kati vaihtaa uraa ja tulee olemaan terapeutti. Q: Kuka tulee olemaan terapeutti? A: Kati.', 
                           'comparative1': 'C: Kati on leveämpi kuin Antti. Q: Kumpi on ohuempi? A: Antti.', 
                           'comparative2': 'C: Jos laatikko olisi leveämpi, Kati hylkäisi sen. Jos pakastin olisi ohuempi, Kati hylkäisi sen. Q: Mitä Kati hylkäisi jos se olisi vähemmän ohuempi? A: Laatikon.',
                           }, 
                    'ja_fam': {'negation1': 'C: 昭夫はプログラマーじゃないけど、文江はそうだ。Q: 誰がプログラマー？A: 結衣と智子。', 
                           'negation2': 'C: 昭夫はプログラマー、文江は療法士だ。Q: プログラマーじゃないのは誰？A: 文江。', 
                           'numerals1': 'C: テーブルの上に一個のいちじくと一個のさくらんぼがある。Q: テーブルの上にはいくつの果物がある？A: 二個。', 
                           'numerals2': 'C: テーブルの上に五個のいちじくと五個のさくらんぼがある。昭夫はいちじくを一個食べた。Q: テーブルの上にいちじくはいくつある？。A: 四個。', 
                           'spatial1': 'C: ボトルは引き出しの上、ボックスは冷凍庫の前にある。Q: ボトルはどこ？A: 引き出しの上。', 
                           'spatial2': 'C: ボトルとボックスは引き出しの上だ。昭夫はボックスを床に置いた。Q: ボトルはどこ？A: 引き出しの上。', 
                           'temporal1': 'C: 昭夫と文江はプログラマーだったけど、文江は転職して今は療法士。 Q: 誰が療法士？A: 文江。', 
                           'temporal2': 'C: 昭夫と文江はプログラマーだけど、文江は転職して療法士になる。Q: 療法士には誰がなるかA: 文江。', 
                           'comparative1': 'C: 昭夫は文江より広い。Q: どっちが薄い？A: 文江。', 
                           'comparative2': 'C: もしその引き出しがもっと広かったら、昭夫はそれを捨てるだろう。もしその冷凍庫がもっと薄かったら、昭夫はそれを捨てるだろう。Q: 何がもっと薄くなかったとしたら、昭夫はそれを捨てるか。A: 引き出し。',
                           'counters': 'C: 引き出しの隣に四つの物があるが、それは包丁だ。Q: 引き出しの隣にいくつの包丁がある？A: 四丁。',},
                     'ja_norm': {'negation1': 'C: 昭夫はプログラマーではないですが、文江はそうです。Q: 誰がプログラマーですか。A: 陽向と陽翔です。', 
                           'negation2': 'C: 昭夫はプログラマー、文江は療法士です。Q: プログラマーでないのは誰ですか。A: 文江です。', 
                           'numerals1': 'C: テーブルの上に十個のいちじくと十個のさくらんぼがあります。Q: テーブルの上にはいくつの果物がありますか。A: 二十個です。', 
                           'numerals2': 'C: テーブルの上に七個のいちじくと九個のさくらんぼがあります。昭夫はいちじくを一個食べました。Q: テーブルの上にいちじくはいくつありますか。A: 六個です。', 
                           'spatial1': 'C: ボトルは引き出しの上、ボックスは冷凍庫の前にあります。Q: ボトルはどこにありますか。A: 引き出しの上です。', 
                           'spatial2': 'C: ボトルとボックスは引き出しの上にあります。昭夫はボックスを床に置きます。Q: ボトルはどこにありますか。A: 引き出しの上。', 
                           'temporal1': 'C: 昭夫と文江はプログラマーでしたが、文江は転職して今は療法士です。 Q: 誰が療法士ですか。A: 文江です。', 
                           'temporal2': 'C: 昭夫と文江はプログラマーですが、文江は転職して療法士になります。Q: 療法士には誰がなりますかA: 文江です。', 
                           'comparative1': 'C: 昭夫は文江より広いです。Q: どちらが薄いですか。A: 文江です。', 
                           'comparative2': 'C: もしその引き出しがもっと広かったら、昭夫はそれを捨てるでしょう。もしその冷凍庫がもっと薄かったら、昭夫はそれを捨てるでしょう。Q: 何がもっと薄くなかったとしたら、昭夫はそれを捨てますか。A: 引き出しです。',
                           'counters': 'C: 引き出しの隣に九つの物がありますが、それらは包丁です。Q: 引き出しの隣にいくつの包丁がありますか。A: 九丁です。',},
                     'ja_form': {'negation1': 'C: 昭夫様はプログラマーではございませんが、文江様はそうでございます。Q: 誰がプログラマーでございますか。A: 文江様でございます。', 
                           'negation2': 'C: 昭夫様はプログラマー、文江様は療法士でございます。Q: プログラマーでないのは誰でございますか。A: 文江様でございます。', 
                           'numerals1': 'C: テーブルの上に二個のいちじくと三個のさくらんぼがございます。Q: テーブルの上にはいくつの果物がございますか。A: 五個ございます。', 
                           'numerals2': 'C: テーブルの上に八個のいちじくと十個のさくらんぼがございます。昭夫はいちじくを二個召し上がりました。Q: テーブルの上にいちじくはいくつございますか。A: 六個ございます。', 
                           'spatial1': 'C: ボトルは引き出しの上、ボックスは冷凍庫の前にございます。Q: ボトルはどこにございますかか。A: 引き出しの上にございます。', 
                           'spatial2': 'C: ボトルとボックスは引き出しの上にございます。昭夫様はボックスを床に置かれました。Q: ボトルはどこにございますか。A: 引き出しの上にございます。', 
                           'temporal1': 'C: 昭夫様と文江様はプログラマーでいらっしゃいましたが、文江様はご転職されて今は療法士でいらっしゃいます。 Q: 誰が療法士でいらっしゃいますか。A: 文江様でございます。', 
                           'temporal2': 'C: 昭夫様と文江様はプログラマーでいらっしゃいますが、文江様はご転職して療法士になられます。Q: 療法士には誰がなられますか。A: 文江様でございます。', 
                           'comparative1': 'C: 昭夫様は文江様より広いでいらっしゃいます。Q: 誰が薄いでいらっしゃいますか？A: 文江様でございます。', 
                           'comparative2': 'C: もしその引き出しがもっと広かったならば、昭夫様はそれを捨てるでしょう。もしその冷凍庫がもっと薄かったならば、昭夫様はそれを捨てるでしょう。Q: 何がもっと薄くなかったならば、昭夫様はそれを捨てるでしょうか。A: 引き出しでございます。',
                           'counters': 'C: 引き出しの隣に七つの物がございますが、それらは包丁でございます。Q: 引き出しの隣にいくつの包丁がございますか。A: 七丁ございます。',}
                }

PREFIX = {'ja_fam': '日本語でお答えて。',
                          'ja_form': '日本語でお答えください。',
                          'ja_norm': '日本語で答えてください。',
                          'en': 'Please answer the question in English.',
                          'de': 'Bitte beantworten Sie die Frage auf Deutsch.',
                          'fi': 'Vastaa kysymykseen suomeksi.'}

In [69]:
import re

def get_targets(prompt, lang, task):
    groups = re.search(TARGET_PATTERN[lang][task], prompt)
    if not groups:
        print(task)
        print(lang)
        print(prompt)
    
    targets = {k: v.lower() for k, v in groups.groupdict().items() if v}
    # if task in {'negation1', 'negation2'}: 
    #     new_items = {}
    #     remove_keys = []
    #     for k, v in targets.items():
    #         if CONJ[lang] in v:
    #             print(v)
    #             v1, v2 = v.split(CONJ[lang] if lang == 'ja' else f' {CONJ[lang]} ')
    #             new_items[k+'A'], new_items[k+'B'] = v1.strip(), v2.strip()
    #             remove_keys.append(k)
    #     targets.update(new_items)
    #     for k in remove_keys:
    #         del targets[k]

    return targets

In [70]:
import json
import os
from collections import Counter
path = '/home/norrman/GitHub/multi-morph-checklist/M2C.json'

if not os.path.isfile(path):
    data = {}
    ind = 0
    for doc in files:
        with open(doc['path'], 'r') as f:
            for line in f:
                context, question, answer = line.strip().replace('C: ', '').replace('Q: ', '|').replace('A: ', '|').split('|')
                lang, task, polite = doc['lang'], doc['task'], doc['polite']
                lang_p = lang + f'_{polite}' if polite else lang

                targets = get_targets(line, lang, task)

                data[ind] = {
                              'task': task,
                              'lang': lang_p,
                              'prefix': PREFIX[lang_p],
                              'context': context,
                              'question': question,
                              'answer': answer,
                              'prompt': f'{PREFIX[lang_p]} C: {context} Q: {question} A:',
                              'targets': targets,
                              'example': ONE_SHOT_PROMPTS[lang_p][task]
                            }
                
                ind += 1
    counts = Counter([d['lang'] for d in data.values()])
    print(counts)
    json.dump(data, open(path, 'w'), indent=4)
else:
    print('ERROR: OUTFILE ALREADY EXISTS')

ERROR: OUTFILE ALREADY EXISTS


In [110]:
task = "counters"

In [113]:
for item in data.values():
    if 'ja_norm' in item['lang'] and item['task'] == task:
        print(item['targets'])

{'P2': '棚の前', 'N1': '九つ', 'P1': '九足'}
{'P2': 'ソファの後ろ', 'N1': '四つ', 'P1': '四本'}
{'P2': '椅子の下', 'N1': '一つ', 'P1': '一冊'}
{'P2': '棚の下', 'N1': '五つ', 'P1': '五本'}
{'P2': 'テーブルの前', 'N1': '七つ', 'P1': '七枚'}
{'P2': 'テーブルの下', 'N1': '八つ', 'P1': '八枚'}
{'P2': '椅子の下', 'N1': '九つ', 'P1': '九冊'}
{'P2': '棚の上', 'N1': '八つ', 'P1': '八冊'}
{'P2': 'ソファの前', 'N1': '二つ', 'P1': '二枚'}
{'P2': '椅子の前', 'N1': '九つ', 'P1': '九本'}
{'P2': 'ソファの上', 'N1': '三つ', 'P1': '三冊'}
{'P2': 'テーブルの下', 'N1': '三つ', 'P1': '三冊'}
{'P2': '椅子の後ろ', 'N1': '四つ', 'P1': '四冊'}
{'P2': 'ソファの下', 'N1': '三つ', 'P1': '三足'}
{'P2': '机の上', 'N1': '九つ', 'P1': '九冊'}
{'P2': '棚の前', 'N1': '二つ', 'P1': '二枚'}
{'P2': '棚の後ろ', 'N1': '二つ', 'P1': '二本'}
{'P2': '机の上', 'N1': '四つ', 'P1': '四冊'}
{'P2': 'テーブルの上', 'N1': '六つ', 'P1': '六冊'}
{'P2': 'テーブルの前', 'N1': '九つ', 'P1': '九本'}
{'P2': 'テーブルの上', 'N1': '二つ', 'P1': '二冊'}
{'P2': 'ソファの前', 'N1': '九つ', 'P1': '九枚'}
{'P2': '机の上', 'N1': '九つ', 'P1': '九冊'}
{'P2': '椅子の前', 'N1': '六つ', 'P1': '六枚'}
{'P2': '椅子の後ろ', 'N1': '九つ', 'P1': '九枚'}
{'P2': '椅子の隣

In [108]:
for item in data.values():
    if 'ja_form' in item['lang'] and item['task'] == task:
        print(item['targets'])

In [109]:
for item in data.values():
    if 'ja_form' in item['lang'] and item['task'] == task:
        print(item['targets'])

## Extracting targets

In [115]:
import re
"""
### NEGATION1
# EN
# {NP} is/are {NEG} {J}, but {P} is/are.
# Who are {J}?
# {P}.

# DE
# {NP} ist/sind {NEG} {J}, aber {P} ist/sind es.'
# Wer ist/sind {J}?
# {P}.

# FI
# {NP} {NEG} ole {J} mutta {P} on/ovat.
# Kuka/Ketkä on/ovat {J}
# {P}.

# JA_FAM
# {NP}は{J}{NEG}ですが、{P}はそうです。
# 何方が{J}ですか。
# {P}です。
"""
"""
### NEGATION2

# EN
# {N1} (is/are) a {N2}, {P1} are {P2}. 
# Who is/are not a? {N2}? 
# {P1}.           

# DE
# {N1} ist/sind ein {N2} und {P1} sind {P2}.
# Wer sind keine {N2}?
# {P1}.

# FI
# {N1} on {N2}, {P1} ovat {P2}. 
# Ketkä eivät ole {N2}? 
# {P1}.

# JA
# {N1}は{N2}、{P1}は{P2}だ。
# {N2}じゃないのは誰？
# {P1}。


# EXP1: P1 in answer
# EXP2: P1 in answer, NEG not in answer
# EXP3: P1 in answer, N1 not in answer
# EXP4: P1 in answer, N2 not in answer
# EXP5: P1 in answer, NEG not in answer, N2 not in answer
# EXP6: P1 in answer, NEG not in answer, N1 not in answer
# EXP7: P1 in answer, N1 not in answer, N2 not in answer
# EXP8: P1 in answer, N1 not in answer, N2 not in answer, NEG not in answer

# EXP1: P1 in answer, P2 in answer,
# EXP2: P1 in answer, P2 in answer, NEG not in answer
# EXP3: P1 in answer, P2 in answer, N1 not in answer
# EXP4: P1 in answer, P2 in answer, N2 not in answer
# EXP5: P1 in answer, P2 in answer, NEG not in answer, N2 not in answer
# EXP6: P1 in answer, P2 in answer, NEG not in answer, N1 not in answer
# EXP7: P1 in answer, P2 in answer, N1 not in answer, N2 not in answer
# EXP8: P1 in answer, P2 in answer, N1 not in answer, N2 not in answer, NEG not in answer
"""
"""
### NUMERALS1
# EN
# There are {N1} {N2} and {N3} {N4} {P3}. 
# How many {P2} in total are {P3}? 
# {P1}.

# DE
# C: Es gibt {N1} {N2} und {N3} {N4} {P3}. 
# Q: Wie viele {P2} liegen insgesamt {P3}? 
# A: {P1}.

# FI
# C: Pöydällä on {N1} {N2} ja {N3} {N4}. 
# Q: Kuinka monta {P2} on {P3} yhteensä? 
# A: {P1}.

# JA
# C: {P3}に{N2}の{N1}と{N4}の{N3}がある。
# Q: {P3}にはいくつの{P2}がある？
# A: {P1}。
"""
"""
### NUMERALS2
# EN
# C: There are {N1A} {P2} and {N1B} {N2} {P3}. Kirk ate {N1C} pineapples. 
# Q: How many pineapples are on the table? 
# A: {P1}.

# DE
# C: Es gibt {N1A} {P2} und {N1B} {N2} {P3}. Katja aß {N1C} Wassermelone. 
# Q: Wie viele Wassermelonen gibt es auf dem Tisch? 
# A: {P1}.

# FI
# C: {P3} on {N1A} {P2} ja {N1B} {N2}. Ylvä söi {N1C} vesimelonia. 
# Q: Kuinka monta vesimelonia on pöydällä? '
# A: {P1}.

# JA
# C: {P3}に{N1A}の{P2}と{N1B}の{N2}がございます。富美子はりんごを二個召し上がりました。
# Q: テーブルの上にりんごはいくつございますか。
# A: {P1}ございます。
"""
"""
### SPATIAL1
# EN
# C: The {N2} is {N1} and the {P2} are {P1}. 
# Q: Where are the phones? 
# A: Next to the shelf.

# DE
# C: Der {N2} liegt {N1} und die {P2} liegen {P1}. 
# Q: Wo sind die Bücher? 
# A: Vor dem Regal.

# FI
# C: {N2} on {N1} ja {P2} {P1}. 
# Q: Missä ovat paperit? 
# A: Tuolin edessä.

# JA
# C: {N2}は{N1}、{P2}は{P1}にございます。
# Q: 紙はどこにございますかか。
# A: 机の上にございます。
"""
"""
### SPATIAL2
# EN
# C: The {P2} and the {N2} are {P1}. Kirk puts the pens {N1}. 
# Q: Where is the book? 
# A: Behind the table.

# DE
# C: Das {P2} und die {N2} liegen {P1}. Judith legt die Papiere {N1}. 
# Q: Wo ist das Buch? 
# A: Vor dem Sofa.

# FI
# C: {P2} ja {N2} ovat {P1}. Okko laittaa paperit {N1}. 
# Q: Missä on kynä? 
# A: Hyllyn päällä.

# JA
# C: {P2}と{N2}はソ{P1}にございます。富美子様は電話を{N1}に置かれました。
# Q: 本はどこにございますか。
# A: ソファの下にございます。
"""
"""
### TEMPORAL1

# EN
# C: {P1} and {N1} were {N2}, but Lola changed careers and is now a {P2}. 
# Q: Who is a farmer? 
# A: Lola.

# DE
# C: {P1} und {N1} waren {N2}, aber Klaus hat den Beruf gewechselt und wird {P2}. 
# Q: Wer ist Manager? 
# A: Klaus.

# FI
# C: {P1} ja {N1} olivat {N2} mutta Hellä vaihtoi uraa ja on nyt {P2}. 
# Q: Kuka on lakimies? 
# A: Hellä.

# JA
# C: {P1}と{N1}は{N2}でいらっしゃいましたが、笑子様はご転職されて今は{P2}でいらっしゃいます。 
# Q: 誰が歌手でいらっしゃいますか。
# A: 笑子様でございます。
"""
"""
### TEMPORAL2

# EN
# C: {P1} and {N1} are {N2}, but Audrey will change careers and will be a {P2}. 
# Q: Who will be a policeman? 
# A: Audrey.

# DE
# C: {P1} und {N1} sind {N2}, aber Günter wechselt den Beruf und wird {P2}. 
# Q: Wer wird Buchhalter? 
# A: Günter.

# FI
# C: {P1} ja {N1} ovat {N2} mutta Jaakko vaihtaa uraa ja tulee olemaan {P2}. 
# Q: Kuka tulee olemaan kirjanpitäjä? 
# A: Jaakko.

# JA
# C: {P1}と{N1}は{N2}でいらっしゃいますが、春花様はご転職して{P2}になられます。
# Q: 警官には誰がなられますか。
# A: 春花様でございます。
"""
"""
### COMPARATIVE1

# EN
# C: {N1} is {N2} than {P1}. 
# Q: Who is {P2}? 
# A: Lola.

# DE
# C: {N1} ist {N2} als {P1}. 
# Q: Wer ist {P2}? 
# A: Stefan.

# FI
# C: {N1} on {N2} kuin {P1}. 
# Q: Kumpi on {P2}? 
# A: Okko.

# JA
# C: {N1}は{P1}より{N2}でいらっしゃいます。
# Q: 誰が{P2}でいらっしゃいますか？
# A: 実様でございます。
"""
"""
### COMPARATIVE2

# EN
# C: If the {P1} was more expensive, Lola would {P2} it. If the {N1} was cheaper, Lola would rent it. 
# Q: What would Lola rent if it was less cheap? 
# A: The house.

# DE
# C: Wäre die {P1} älter, würde sie Stefan {P2}. Wäre das {N1} teurer, würde es Stefan mieten. 
# Q: Was würde Stefan mieten, wäre es weniger neu? 
# A: Die Wohnung.

# FI
# C: Jos kirja olisi pienempi, Jaakko {P2} sitä. Jos {N1} olisi isompi, Jaakko käyttäisi sitä. 
# Q: Mitä Jaakko käyttäisi jos se olisi vähemmän iso? 
# A: {P1}.

# JA
# C: もしその{P1}がもっと美しかったならば、龍様はそれを{P2}でしょう。もしその{N1}がもっと大きかったならば、龍様はそれを売るでしょう。
# Q: 何がもっと醜くなかったならば、龍様はそれを売るでしょうか。
# A: 船でございます。
"""
"""
### COUNTERS
# JA
# C: 棚の上に{N1}の物がありますが、それらは辞書です。
# Q: 棚の上にいくつの辞書がありますか。
# A: {P1}です。
"""
input_string = {"negation1": {"en": "C: Please answer the question in English. C: Susan is not an architect, but Lola is.  Q: Who is an architect?  A:",
                              "de": "C: Heike und Fredrik ist keine Managerin, aber Johann und Karin ist es.",
                              "fi": "C: Hilppa ja Henriikka eivät ole poliiseja mutta Kaija ja Juuli ovat.",
                              "ja": "C: 陽向と陽翔は弁護士ではないですが、誠と清はそうです。"},

                "negation2": {"en": "C: Steve is an engineer, Christopher and Charles are lawyers.",
                              "de": "C: Günter ist ein Kellner und Hilda und Gretel sind Wissenschaftlerinnen.",
                              "fi": "C: Jaakko on kirjanpitäjä, Säde ja Matleena ovat viljelijöitä.",
                              "ja": "C: 希実様と明美様は監督、明梨様と優花様は技師でございます。Q: 監督でないのは誰でございますか。A: 明梨様と優花様でございます。"},

                "numerals1": {"en": "C: There are two pineapples and six mangoes on the table. Q: How many fruits in total are on the table? A: Eight.",
                              "de": "C: Es gibt eine Mango und zwei Äpfel auf dem Tisch. Q: Wie viele Früchte liegen insgesamt auf dem Tisch? A: Drei.",
                              "fi": "C: Pöydällä on yksi kirsikka ja yksi mansikka. Q: Kuinka monta hedelmää on pöydällä yhteensä? A: Kaksi.",
                              "ja": "C: テーブルの上に七個のスイカと七個のみかんがございます。Q: テーブルの上にはいくつの果物がございますか。A: 十四個ございます。"},

                "numerals2": {"en": "C: There are six pineapples and six strawberries on the table. Kirk ate two pineapples. Q: How many pineapples are on the table? A: Four.",
                              "de": "C: Es gibt acht Wassermelonen und neun Äpfel auf dem Tisch. Katja aß eine Wassermelone. Q: Wie viele Wassermelonen gibt es auf dem Tisch? A: Sieben.",
                              "fi": "C: Pöydällä on kuusi vesimelonia ja yhdeksän banaania. Ylvä söi nolla vesimelonia. Q: Kuinka monta vesimelonia on pöydällä? A: Kuusi.",
                              "ja": "C: テーブルの上に五個のりんごと六個の桃がございます。富美子はりんごを二個召し上がりました。Q: テーブルの上にりんごはいくつございますか。A: 三個ございます。"},
                
                "spatial1": {"en": "C: The mug is behind the window and the phones are next to the shelf. Q: Where are the phones? A: Next to the shelf.",
                              "de": "C: Der Rechner liegt hinter dem Sofa und die Bücher liegen vor dem Regal. Q: Wo sind die Bücher? A: Vor dem Regal.",
                              "fi": "C: Kynä on ikkunan vieressä ja paperit tuolin edessä. Q: Missä ovat paperit? A: Tuolin edessä.",
                              "ja": "C: 紙は机の上、電話は棚の上にございます。Q: 紙はどこにございますかか。A: 机の上にございます。"},
                
                "spatial2": {"en": "C: The book and the pens are behind the table. Kirk puts the pens on the ground. Q: Where is the book? A: Behind the table.",
                              "de": "C: Das Buch und die Papiere liegen vor dem Sofa. Judith legt die Papiere auf den Boden. Q: Wo ist das Buch? A: Vor dem Sofa.",
                              "fi": "C: Kynä ja paperit ovat hyllyn päällä. Okko laittaa paperit maahan. Q: Missä on kynä? A: Hyllyn päällä.",
                              "ja": "C: 本と電話はソファの下にございます。富美子様は電話を床に置かれました。Q: 本はどこにございますか。A: ソファの下にございます。"},
                "temporal1": {"en": "C: Lola and Audrey were writers, but Lola changed careers and is now a farmer. Q: Who is a farmer? A: Lola.",
                              "de": "C: Ilse und Judith waren Polizistinnen, aber Ilse hat den Beruf gewechselt und wird Architektin. Q: Wer ist Architekt? A: Ilse.",
                              "fi": "C: Hellä ja Tuulikki olivat tarjoilijoita mutta Hellä vaihtoi uraa ja on nyt lakimies. Q: Kuka on lakimies? A: Hellä.",
                              "ja": "C: 笑子様と龍様は教師でいらっしゃいましたが、笑子様はご転職されて今は歌手でいらっしゃいます。 Q: 誰が歌手でいらっしゃいますか。A: 笑子様でございます。"},
                "temporal2": {"en": "C: Audrey and Clark are writers, but Audrey will change careers and will be a policeman. Q: Who will be a policeman? A: Audrey.",
                              "de": "C: Ilse und Leonie sind Wissenschaftlerinnen, aber Ilse wechselt den Beruf und wird Bäuerin. Q: Wer wird Bauer? A: Ilse.",
                              "fi": "C: Jaakko ja Päiviö ovat lakimiehiä mutta Jaakko vaihtaa uraa ja tulee olemaan kirjanpitäjä. Q: Kuka tulee olemaan kirjanpitäjä? A: Jaakko.",
                              "ja": "C: 春花様と大樹様は歯科医でいらっしゃいますが、春花様はご転職して警官になられます。Q: 警官には誰がなられますか。A: 春花様でございます。"},
                "comparative1": {"en": "C: Kirk is taller than James. Q: Who is shorter? A: James.",
                              "de": "C: Günter ist langsamer als Stefan. Q: Wer ist schneller? A: Stefan.",
                              "fi": "C: Eero on iloisempi kuin Okko. Q: Kumpi on surullisempi? A: Okko.",
                              "ja": "C: 笑子様は実様より大きいでいらっしゃいます。Q: 誰が小さいでいらっしゃいますか？A: 実様でございます。"},   
                "comparative2": {"en": "C: If the house was more expensive, Lola would rent it. If the bike was cheaper, Lola would rent it. Q: What would Lola rent if it was less cheap? A: The house.",
                              "de": "C: Wäre die Wohnung älter, würde sie Stefan mieten. Wäre das Auto teurer, würde es Stefan mieten. Q: Was würde Stefan mieten, wäre es weniger neu? A: Die Wohnung.",
                              "fi": "C: Jos kirja olisi pienempi, Jaakko käyttäisi sitä. Jos tietokone olisi isompi, Jaakko käyttäisi sitä. Q: Mitä Jaakko käyttäisi jos se olisi vähemmän iso? A: Kirjaa.",
                              "ja": "C: もしその船がもっと美しかったならば、龍様はそれを売るでしょう。もしそのパソコンがもっと大きかったならば、龍様はそれを売るでしょう。Q: 何がもっと醜くなかったならば、龍様はそれを売るでしょうか。A: 船でございます。"},  
                "counters": {"ja": "C: 棚の上に六つの物がありますが、それらは辞書です。Q: 棚の上にいくつの辞書がありますか。A: 六冊です。"}             
                              }

lang = 'ja'
task = 'counters'

# Define the regular expression pattern
# Use re.search to find the match
match = re.search(TARGET_PATTERN[lang][task], input_string[task][lang])
print(input_string[task][lang])
print(TARGET_PATTERN[lang][task])
if match:
    # Print the extracted components
    
    print(match.groupdict())


C: 棚の上に六つの物がありますが、それらは辞書です。Q: 棚の上にいくつの辞書がありますか。A: 六冊です。
.*C\:\s(?P<P2>.+?)に(?P<N1>.+?)の物.*A:\s(?P<P1>.+?)(?:。|です。|ございます。).*
{'P2': '棚の上', 'N1': '六つ', 'P1': '六冊'}
